In [14]:
import pandas as pd
import helpers

from IPython.display import display
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
# from sklearn.metrics import log_loss
# from sklearn.naive_bayes import BernoulliNB
# from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import KFold
import numpy as np

# convert the Dates column of our provided data from string to datetime format.
train=pd.read_csv('train.csv', parse_dates = ['Dates'])
test=pd.read_csv('test.csv', parse_dates = ['Dates'])
training, validation = train_test_split(train, train_size=.60)

# Print the first 5 rows of the dataframe.
print train.head(2)


                Dates        Category                  Descript  DayOfWeek  \
0 2015-05-13 23:53:00        WARRANTS            WARRANT ARREST  Wednesday   
1 2015-05-13 23:53:00  OTHER OFFENSES  TRAFFIC VIOLATION ARREST  Wednesday   

  PdDistrict      Resolution             Address           X          Y  
0   NORTHERN  ARREST, BOOKED  OAK ST / LAGUNA ST -122.425892  37.774599  
1   NORTHERN  ARREST, BOOKED  OAK ST / LAGUNA ST -122.425892  37.774599  


In [15]:
def prep(data):
    days = pd.get_dummies(data.DayOfWeek)
    district = pd.get_dummies(data.PdDistrict)
    
    # Gets the hour portion form the "Dates" column
    hour = data.Dates.dt.hour
    
    #Build new array with binary info on location, day and hour and numbered crime
    train_data = pd.concat([hour, days, district], axis=1)
    return train_data


train_data = prep(training)
# 878049 x 18 columns
display(train_data.head())

,Dates,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,BAYVIEW,CENTRAL,INGLESIDE,MISSION,NORTHERN,PARK,RICHMOND,SOUTHERN,TARAVAL,TENDERLOIN
287724,19,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
388098,23,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
842064,17,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0
90899,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
600781,16,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [16]:
#Convert crime labels to numbers
le_crime = preprocessing.LabelEncoder()
encodedCrime = le_crime.fit_transform(training.Category)
humanReadableCrime = le_crime.inverse_transform(encodedCrime)
print humanReadableCrime
print encodedCrime

['SUSPICIOUS OCC' 'VANDALISM' 'VEHICLE THEFT' ..., 'VEHICLE THEFT'
 'LARCENY/THEFT' 'BURGLARY']
[32 35 36 ..., 36 16  4]


In [17]:
from sklearn import tree
X = train_data
Y = encodedCrime
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, Y)

In [18]:
validation_data = prep(validation)
validationPredictions = clf.predict(validation_data)

In [19]:
humanReadableValidationPredictions = le_crime.inverse_transform(validationPredictions)

In [20]:
validationTrueLabels = le_crime.fit_transform(validation.Category)
display(validation.head())
print(le_crime.inverse_transform(validationPredictions[0:5]))

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
17079,2015-02-20 01:00:00,MISSING PERSON,MISSING ADULT,Friday,MISSION,NONE,2000 Block of MARKET ST,-122.426956,37.769247
70679,2014-06-09 21:30:00,LARCENY/THEFT,GRAND THEFT FROM UNLOCKED AUTO,Monday,TENDERLOIN,NONE,OFARRELL ST / HYDE ST,-122.416262,37.785375
833295,2003-08-07 18:05:00,BURGLARY,"BURGLARY OF STORE, FORCIBLE ENTRY",Thursday,SOUTHERN,NONE,1600 Block of MISSION ST,-122.419794,37.770363
626898,2006-06-26 07:48:00,FRAUD,FRAUDULENT CREDIT APPLICATION,Monday,CENTRAL,NOT PROSECUTED,2500 Block of MASON ST,-122.413814,37.807036
790506,2004-03-05 14:00:00,VEHICLE THEFT,"VEHICLE, RECOVERED, AUTO",Friday,MISSION,NONE,SANCHEZ ST / 17TH ST,-122.430726,37.762752


['LARCENY/THEFT' 'DRUG/NARCOTIC' 'LARCENY/THEFT' 'LARCENY/THEFT'
 'LARCENY/THEFT']


In [21]:
print validationPredictions[0:10]
print validationTrueLabels[0:10]
counter = 0
for i in range(len(validationPredictions)):
    if validationPredictions[i] == validationTrueLabels[i]:
        counter+=1

accuracy = counter/float(len(validationPredictions))
print accuracy

[16  7 16 16 16 16 21 21 21 16]
[19 16  4 13 36 19 20 16 21 13]
0.22314219008


In [23]:
from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier

preppedTrainData = prep(train)
alg = RandomForestClassifier(random_state=1, n_estimators=10, min_samples_split=4, min_samples_leaf=1)
scores = cross_validation.cross_val_score(alg, preppedTrainData, train["Category"], cv=3)

print scores.mean()

0.22072684747


After testing both a simple decision tree and a RandomForrest algorith from sklearn we have resulted with a pretty low accuracy score. We believe that the one-hot-encoding method we used (prep()) to encode categorical data, such as days of the week, did not yeild interesting results in the accuracy predictions. 